<a href="https://colab.research.google.com/github/aimedvedeva/Competition2-nlp-/blob/main/comp2_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd

prompts_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Comp2/prompts_train.csv")
summaries_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Comp2/summaries_train.csv")

In [15]:
len(prompts_df)

4

In [16]:
len(summaries_df)

7165

In [17]:
combined_train_df = pd.merge(summaries_df, prompts_df, how = 'left', on = 'prompt_id')

In [3]:
assert len(combined_train_df) == len(summaries_df)

In [4]:
combined_train_df.columns

Index(['student_id', 'prompt_id', 'text', 'content', 'wording',
       'prompt_question', 'prompt_title', 'prompt_text'],
      dtype='object')

In [5]:
combined_train_df.describe()

,content,wording
count,7165.000000,7165.000000
mean,-0.014853,-0.063072
std,1.043569,1.036048
min,-1.729859,-1.962614
25%,-0.799545,-0.872720
50%,-0.093814,-0.081769
75%,0.499660,0.503833
max,3.900326,4.310693


In [6]:
combined_train_df[(combined_train_df['prompt_id'] == 'ebad26') & (combined_train_df['content'] < -1.4)].iloc[0]['text']

'They would "rub it up with soda to take away the smell, pickle it with a "stronger pickle" to remove the smell, and they would extract the bone and cook the inside to sell them as "boneless hams".'

In [7]:
combined_train_df[(combined_train_df['prompt_id'] == 'ebad26') & (combined_train_df['content'] > 3.4)].iloc[0]['text']

'There were various things that the factory would do to cover up and essentially use the spoiled meat that they had. One manner in which they practiced this was by rubbing the meat with soda to take away the smell, as stated in paragraph two.  The workers would also pickle their meat to get rid of the odor but if it was very pungent, they had a second and much stronger pickle which helped destroy the smell, which the workers called, "giving them thirty per cent." Along with this, they would label the packages in many ways to make it look like it was "original" even if they were made out of meat parts that were not supposed to be used. Another form in which they would use and cover-up spoiled meat would be by getting spoiled whole ham and old sausage with mold from Europe and dosing it with borax and glycerin, where it would then be dumped into hoppers and made over again for packaging. Mice, feces, poisoned bread, dirt, diseases, and sawdust were just a few things that would be process

# Calculate metrics

In [19]:
summaries = combined_train_df['text'].tolist()
prompt_texts = combined_train_df['prompt_text'].tolist()
prompt_questions = combined_train_df['prompt_question'].tolist()

In [18]:
combined_train_df.columns

Index(['student_id', 'prompt_id', 'text', 'content', 'wording',
       'prompt_question', 'prompt_title', 'prompt_text'],
      dtype='object')

#### [BERTScore](https://huggingface.co/spaces/evaluate-metric/bertscore)


> BERTScore leverages the pre-trained contextual embeddings from BERT and matches words in candidate and reference sentences by cosine similarity. It has been shown to correlate with human judgment on sentence-level and system-level evaluation. Moreover, BERTScore computes precision, recall, and F1 measure, which can be useful for evaluating different language generation tasks.



##### pips

In [ ]:
pip install evaluate

In [ ]:
pip install bert_score

##### calculation

In [7]:
from evaluate import load
bertscore = load("bertscore")

In [ ]:
bert_scores = []
for i in range(len(summaries)):
  print(i)
  score = bertscore.compute(predictions=[summaries[i]], references=[prompt_texts[i]], lang="en")
  print(score)
  bert_scores.append(score)

0
{'precision': [0.8283703923225403], 'recall': [0.7932977080345154], 'f1': [0.8104547262191772], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.31.0)'}
1
{'precision': [0.8546106219291687], 'recall': [0.7895169258117676], 'f1': [0.8207751512527466], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.31.0)'}
2
{'precision': [0.8588669896125793], 'recall': [0.8438898324966431], 'f1': [0.851312518119812], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.31.0)'}
3
{'precision': [0.8455113768577576], 'recall': [0.7886843085289001], 'f1': [0.816109836101532], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.31.0)'}
4
{'precision': [0.8321478366851807], 'recall': [0.8212687969207764], 'f1': [0.8266725540161133], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.31.0)'}
5
{'precision': [0.8613998889923096], 'recall': [0.7893351912498474], 'f1': [0.8237944841384888], 'hashcode': 'roberta-large_L17_no-idf_version=0.3

In [ ]:
precisions = bert_scores['precision']
recalls = bert_scores['recall']
f1s = bert_scores['f1']

#### [CharacterScore](https://huggingface.co/spaces/evaluate-metric/character)
> CharacTer is a character-level metric inspired by the translation edit rate (TER) metric. It is defined as the minimum number of character edits required to adjust a hypothesis, until it completely matches the reference, normalized by the length of the hypothesis sentence. CharacTer calculates the character level edit distance while performing the shift edit on word level. Unlike the strict matching criterion in TER, a hypothesis word is considered to match a reference word and could be shifted, if the edit distance between them is below a threshold value. The Levenshtein distance between the reference and the shifted hypothesis sequence is computed on the character level. In addition, the lengths of hypothesis sequences instead of reference sequences are used for normalizing the edit distance, which effectively counters the issue that shorter translations normally achieve lower TER.

In [ ]:
import evaluate
character = evaluate.load("character")

In [ ]:
character_scores = character.compute(references=prompt_texts, predictions=summaries)

In [ ]:
cer_scores = character_scores['cer_scores']

#### [NIST_MT](https://huggingface.co/spaces/evaluate-metric/nist_mt)
> DARPA commissioned NIST to develop an MT evaluation facility based on the BLEU score. The official script used by NIST to compute BLEU and NIST score is mteval-14.pl. The main differences are:
* BLEU uses geometric mean of the ngram overlaps, NIST uses arithmetic mean.
* NIST has a different brevity penalty
* NIST score from mteval-14.pl has a self-contained tokenizer (in the Hugging Face implementation we rely on NLTK’s implementation of the NIST-specific tokenizer)

In [ ]:
import evaluate
nist_mt = evaluate.load("nist_mt")

In [ ]:
nist_scores = nist_mt.compute(preds=summaries, refs=prompt_texts)
nist_mt_scores = nist_scores['nist_mt']

#### [Charcut](https://huggingface.co/spaces/evaluate-metric/charcut_mt)
> CharCut compares outputs of MT systems with reference translations. The matching algorithm is based on an iterative search for longest common substrings, combined with a length-based threshold that limits short and noisy character matches. As a similarity metric this is not new, but to the best of our knowledge it was never applied to highlighting and scoring of MT outputs. It has the neat effect of keeping character-based differences readable by humans

In [ ]:
import evaluate
charcut = evaluate.load("charcut")

In [ ]:
results = charcut.compute(references=prompt_texts, predictions=summaries)
charcut_scores = results['charcut_mt']

#### [ExactMatch](https://huggingface.co/spaces/evaluate-metric/exact_match)
> A given predicted string’s exact match score is 1 if it is the exact same as its reference string, and is 0 otherwise.
* Example 1: The exact match score of prediction “Happy Birthday!” is 0, given its reference is “Happy New Year!”.
* Example 2: The exact match score of prediction “The Colour of Magic (1983)” is 1, given its reference is also “The Colour of Magic (1983)”.
The exact match score of a set of predictions is the sum of all of the individual exact match scores in the set, divided by the total number of predictions in the set.
* Example: The exact match score of the set {Example 1, Example 2} (above) is 0.5.



In [ ]:
from evaluate import load
exact_match_metric = load("exact_match")

In [ ]:
results = exact_match_metric.compute(predictions=summaries, references=prompt_texts)
exact_match_scores = results['exact_match']

#### [RougeScore](https://huggingface.co/spaces/evaluate-metric/rouge)

> ROUGE, or Recall-Oriented Understudy for Gisting Evaluation, is a set of metrics and a software package used for evaluating automatic summarization and machine translation software in natural language processing. The metrics compare an automatically produced summary or translation against a reference or a set of references (human-produced) summary or translation.
Note that ROUGE is case insensitive, meaning that upper case letters are treated the same way as lower case letters.

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
results = rouge.compute(predictions=summaries, references=prompt_texts, use_aggregator=False)

rouge1_scores = results['rouge1']
rouge2_scores = results['rouge2']
rougeL_scores = results['rougeL']
rougeLsum_scores = results['rougeLsum']

#### [CometScore](https://huggingface.co/spaces/evaluate-metric/wer)


> Crosslingual Optimized Metric for Evaluation of Translation (COMET) is an open-source framework used to train Machine Translation metrics that achieve high levels of correlation with different types of human judgments.

*COMET takes 3 lists of strings as input: sources (a list of source sentences), predictions (a list of candidate translations) and references (a list of reference translations).*

In [ ]:
from evaluate import load
comet_metric = load('comet')

In [ ]:
results = comet_metric.compute(predictions=summaries, references=prompt_questions, sources=prompt_texts)
comet_scores = [round(v, 2) for v in results["scores"]]

#### [MAUVE](https://huggingface.co/spaces/evaluate-metric/mauve)
> MAUVE is a measure of the gap between neural text and human text. It is computed using the Kullback–Leibler (KL) divergences between the two distributions of text in a quantized embedding space of a large language model. MAUVE can identify differences in quality arising from model sizes and decoding algorithms.



In [ ]:
from evaluate import load
mauve = load('mauve')

In [ ]:
mauve_results = []

for i in range(len(summaries)):
  mauve_results.append(mauve.compute(predictions=[summaries[i]], references=[prompt_texts[i]]))

In [ ]:
mauve_scores = [item['mauve'] for item in mauve_results]

#### [MeteorScore](https://huggingface.co/spaces/evaluate-metric/meteor)

> METEOR (Metric for Evaluation of Translation with Explicit ORdering) is a machine translation evaluation metric, which is calculated based on the harmonic mean of precision and recall, with recall weighted more than precision.
METEOR is based on a generalized concept of unigram matching between the machine-produced translation and human-produced reference translations. Unigrams can be matched based on their surface forms, stemmed forms, and meanings. Once all generalized unigram matches between the two strings have been found, METEOR computes a score for this matching using a combination of unigram-precision, unigram-recall, and a measure of fragmentation that is designed to directly capture how well-ordered the matched words in the machine translation are in relation to the reference.



In [ ]:
meteor = evaluate.load('meteor')

In [ ]:
meteor_scores = []

for i in range(len(summaries)):
  meteor_scores.append(meteor.compute(predictions=[summaries[i]], references=[prompt_texts[i]]))

In [ ]:
meteor_scores = [item['meteor'] for item in meteor_scores]

#### [SariScore](https://huggingface.co/spaces/evaluate-metric/sari)


> SARI (system output against references and against the input sentence) is a metric used for evaluating automatic text simplification systems.
The metric compares the predicted simplified sentences against the reference and the source sentences. It explicitly measures the goodness of words that are added, deleted and kept by the system.

SARI can be computed as:

sari = ( F1_add + F1_keep + P_del) / 3

where

F1_add is the n-gram F1 score for add operations

F1_keep is the n-gram F1 score for keep operations

P_del is the n-gram precision score for delete operations

The number of n grams, n, is equal to 4, as in the original paper.



In [ ]:
from evaluate import load
sari = load("sari")

The metric takes 3 inputs: sources (a list of source sentence strings), predictions (a list of predicted sentence strings) and references (a list of lists of reference sentence strings)

In [ ]:
sari_scores = []
for i in range(len(summaries)):
  sari_scores.append(sari.compute(sources=prompt_texts, predictions=summaries, references=prompt_questions))

sari_scores = [item['sari'] for item in sari_scores]

#### SQUAD/CUAD TODO

[CuadScore](https://huggingface.co/spaces/evaluate-metric/cuad)


> This metric wraps the official scoring script for version 1 of the Contract Understanding Atticus Dataset (CUAD), which is a corpus of more than 13,000 labels in 510 commercial legal contracts that have been manually labeled to identify 41 categories of important clauses that lawyers look for when reviewing contracts in connection with corporate transactions.
The CUAD metric computes several scores: Exact Match, F1 score, Area Under the Precision-Recall Curve, Precision at 80% recall and Precision at 90% recall.



In [ ]:
from evaluate import load
cuad_metric = load("cuad")

**The CUAD metric takes two inputs :**

*predictions*, a list of question-answer dictionaries with the following key-values:

* id: the id of the question-answer pair as given in the references.
* prediction_text: a list of possible texts for the answer, as a list of strings depending on a threshold on the confidence probability of each prediction.

*references*: a list of question-answer dictionaries with the following key-values:

* id: the id of the question-answer pair (the same as above).
answers: a dictionary in the CUAD dataset format with the following keys:
* text: a list of possible texts for the answer, as a list of strings.
answer_start: a list of start positions for the answer, as a list of ints.
Note that answer_start values are not taken into account to compute the metric.

In [ ]:
predictions = [{'prediction_text': ['The seller:', 'The buyer/End-User: Shenzhen LOHAS Supply Chain Management Co., Ltd.'], 'id': 'LohaCompanyltd_20191209_F-1_EX-10.16_11917878_EX-10.16_Supply Agreement__Parties'}]
references = [{'answers': {'answer_start': [143, 49], 'text': ['The seller:', 'The buyer/End-User: Shenzhen LOHAS Supply Chain Management Co., Ltd.']}, 'id': 'LohaCompanyltd_20191209_F-1_EX-10.16_11917878_EX-10.16_Supply Agreement__Parties'}]
results = cuad_metric.compute(predictions=predictions, references=references)
print(results)
{'exact_match': 100.0, 'f1': 100.0, 'aupr': 0.0, 'prec_at_80_recall': 1.0, 'prec_at_90_recall': 1.0}

#### Build the final dataset

In [ ]:
dataset = pd.DataFrame(columns = ['precision', 'recall', 'f1', 'cer_score', 'nist_mt_score', 'charcut_mt_score', 'exact_match_score',\
                                  'rouge1_score', 'rouge2_score', 'rougeL_score', 'rougeLsum_score', 'comet_score'\
                                  'mauve_score', 'meteor', 'sari', 'content', 'wording'], data=[])
# all columns except wording and content are features
# content and wording are target values
# we gonna train a regression model to predict content and wording

In [ ]:
dataset['precision'] = precisions
dataset['recall'] = recalls
dataset['f1'] = f1s
dataset['cer_score'] = cer_scores
dataset['nist_mt_score'] = nist_mt_scores
dataset['charcut_mt_score'] = charcut_scores
dataset['exact_math_score'] = exact_match_scores
dataset['rouge1'] = rouge1_scores
dataset['rouge2'] = rouge2_scores
dataset['rougeL'] = rougeL_scores
dataset['rougeLsum'] = rougeLsum_scores
dataset['rouge1'] = rouge1_scores
dataset['comet_score'] = comet_scores
dataset['mauve_score'] = mauve_scores
dataset['meteor'] = meteor_scores
dataset['sari'] = sari_scores
dataset['content'] = combined_train_df['content']
dataset['wording'] = combined_train_df['wording']